In [38]:
# !pip install tensorflow networkx pysmiles deepchem
from pysmiles import read_smiles
from networkx.algorithms.approximation import treewidth_min_degree
from networkx.algorithms.approximation import treewidth_min_fill_in
from networkx.algorithms import junction_tree
import networkx as nx
import itertools
import math
import copy
import time

In [39]:
pcm = read_smiles("CC(=O)Nc1ccc(O)cc1")

In [40]:
def remap_graph(graph: nx.Graph) -> nx.Graph:
  """Takes an input graph and remaps it such that the old key/value pair of a node turns into (integer, old key)
  """
  
  #remap nodes
  node_mapping = {}
  iter = 0
  for node in graph.nodes:
    # print(f"In remap, setting {node} equal to {iter}")
    node_mapping[node] = iter
    iter += 1

  #remap edges
  new_edges = []
  for pair in graph.edges:
    new_edges.append((node_mapping[pair[0]],node_mapping[pair[1]]))
  output_graph = nx.Graph()
  for node in node_mapping:
    # print(node,node_mapping[node])
    output_graph.add_node(node_mapping[node],values=list(node))
  output_graph.add_edges_from(new_edges)
  return output_graph

In [41]:

def add_leaves(decomposition):
  """ Adds nice tree decomposition-defined "leaves" to input graph (which is assumed to be a tree decomposition here)

  The definition of a leaf in a nice tree decomposition is that it contains only
  one element as its value. 
  This function finds all terminal vertices in the graph and counts the number
  of values they each contain. If any terminal vertex contains more than 1 value,
  a leaf is created by appending a node to be the new terminal vertex and setting
  the value of this node equal to one value of the previously terminal vertex.
  Returns a new graph with leaves added as well as a new root which is the first leaf added 
  E.g.:   (1,2,3) - (2,3)

  Would construct leaves on both sides as:

  (1,) - (1,2,3) - (2,3) - (2,)
  """

  output = decomposition.copy()
  temporary_root = None
  values = nx.get_node_attributes(decomposition, "values")
  current_id = max(output.nodes)
  for node, degree in list(output.degree()):
    # print(node, degree)
    if degree == 0 and len(values[node]) > 1:
      current_id += 1
      output.add_node(current_id,values=[values[node][0],])
      output.add_edge(node,current_id)
      current_id += 1
      output.add_node(current_id,values=[values[node][1],])
      output.add_edge(node,current_id)
      if temporary_root == None:
        temporary_root = current_id
      

    elif degree == 1 and len(values[node]) > 1:
      current_id += 1
      output.add_node(current_id,values=[values[node][0],])
      output.add_edge(node,current_id)
      if temporary_root == None:
        # We s
        temporary_root = current_id
  output.add_node(-1,values=[])
  output.add_edge(-1,temporary_root)
  return output, -1

In [42]:
# Constructs junction tree (decomposition), "remaps" the decomposition values
# and adds leaves
def construct_junction_and_add_leaves(graph):
  remapped_and_leaved_graph, root = add_leaves(remap_graph(junction_tree(graph)))
  return remapped_and_leaved_graph, root

In [43]:
pcm_graph, pcm_root = construct_junction_and_add_leaves(pcm)

In [44]:
pcm_graph.nodes(data=True)

NodeDataView({0: {'values': [3, 4]}, 1: {'values': [6, 7, 9]}, 2: {'values': [1, 2]}, 3: {'values': [0, 1]}, 4: {'values': [5, 6, 10]}, 5: {'values': [7, 8]}, 6: {'values': [4, 5, 10]}, 7: {'values': [1, 3]}, 8: {'values': [6, 9, 10]}, 9: {'values': [4]}, 10: {'values': [3]}, 11: {'values': [6, 9]}, 12: {'values': [7]}, 13: {'values': [1]}, 14: {'values': [5, 10]}, 15: {'values': [6, 10]}, 16: {'values': [1]}, 17: {'values': [0]}, 18: {'values': [7]}, -1: {'values': []}})

In [45]:
def handle_promiscuous_node2(decomposition, promiscuous_node, parent):
  
  new_bag = decomposition.nodes.data("values")[promiscuous_node]
  new_node_id = max(decomposition.nodes)+1
  # print("adding new bag: ", new_bag)
  decomposition.add_node(new_node_id,values=new_bag)
  # print("removing parent: ",parent)
  # print(list(decomposition.neighbors(promiscuous_node)))
  neighbors = list(decomposition.neighbors(promiscuous_node))
  # print("children to update: ",children_to_update)
  if parent != None:
    neighbors.remove(parent)
  children_to_update = neighbors[1::]
  decomposition.add_edge(promiscuous_node,new_node_id)
  for child in children_to_update:
    decomposition.remove_edge(promiscuous_node,child)
    decomposition.add_edge(new_node_id,child)


def handle_promiscuous_nodes2(decomposition,root,parent):
  # print("current root: ",root)
  # print("children: ",children)
  children = list(decomposition.neighbors(root))
  # print("in handle prom. nodes. children: ",children)
  if parent != None:
    children.remove(parent)
  if len(children) > 2:
    handle_promiscuous_node2(decomposition,root,None)
    handle_promiscuous_nodes2(decomposition,root,None)
  else:
    for child in children:
      if len(list(decomposition.neighbors(child))) > 3:
        handle_promiscuous_node2(decomposition,child,root)
      handle_promiscuous_nodes2(decomposition,child,root)


In [46]:
handle_promiscuous_nodes2(pcm_graph,pcm_root,None)

# Has not been tested with a graph actually containing promiscuous nodes yet - maybe such nodes will never be constructed by the junction tree function? Test it out.

In [47]:
def insert_bridging_node(graph, root, child, new_node_id, new_node_values:list):
  assert(type(new_node_values) == list)
  graph.add_node(new_node_id,values=new_node_values)
  graph.add_edge(root,new_node_id)
  graph.add_edge(new_node_id,child)
  graph.remove_edge(root,child)

In [48]:
def handle_join_node(decomposition,root,children):
  """This function handles the construction of a single join node and its two children.
  The join node is given in the `root` variable and the two children are given in the
  `children` variable.
  The join operation is performed by taking the union of the values of all three
  nodes: the parent and its two children, and then setting the values of all three
  nodes equal to this unioned bag.
  """
  # print("Handling join node: ", root)
  assert(len(children) == 2)
  assert(type(children) == list)
  values = nx.get_node_attributes(decomposition, "values")
  root_values = set(values[root])
  child0_values = set(values[children[0]])
  child1_values = set(values[children[1]])
  if root_values == child0_values == child1_values:
    # print(f"Join node {root} already fulfills join identity")
    return children[0], children[1]
  unioned_bag = set(values[root]) | set(values[children[0]]) | set(values[children[1]])
  new_node_id_1 = max(decomposition.nodes)+1
  new_node_id_2 = new_node_id_1+1
  insert_bridging_node(decomposition, root, children[0], new_node_id_1,list(unioned_bag))
  insert_bridging_node(decomposition, root, children[1], new_node_id_2,list(unioned_bag))
  # print("unioned bag:")
  # print(unioned_bag)
  decomposition.nodes[root]["values"] = list(unioned_bag)
  return new_node_id_1, new_node_id_2
  # decomposition.nodes[children[0]]["values"] = tuple(unioned_bag)
  # decomposition.nodes[children[1]]["values"] = tuple(unioned_bag)

In [49]:
def handle_join_nodes(decomposition,root,parent):
  children = list(decomposition.neighbors(root))
  # print(children)
  if parent != None:
    children.remove(parent)
  if len(children) == 2:
    new_child_1, new_child_2 = handle_join_node(decomposition,root,children)
    handle_join_nodes(decomposition,new_child_1,root)
    handle_join_nodes(decomposition,new_child_2,root)
  else:
    for child in children:
      handle_join_nodes(decomposition,child,root)

In [50]:
pcm_graph, pcm_root = construct_junction_and_add_leaves(pcm)
handle_promiscuous_nodes2(pcm_graph,pcm_root,None)
handle_join_nodes(pcm_graph,pcm_root,None)

In [51]:
pcm_graph.nodes(data=True)

NodeDataView({0: {'values': [3, 4]}, 1: {'values': [6, 7, 9]}, 2: {'values': [1, 2]}, 3: {'values': [0, 1]}, 4: {'values': [5, 6, 10]}, 5: {'values': [7, 8]}, 6: {'values': [4, 5, 10]}, 7: {'values': [1, 3]}, 8: {'values': [6, 9, 10]}, 9: {'values': [4]}, 10: {'values': [3]}, 11: {'values': [6, 9]}, 12: {'values': [7]}, 13: {'values': [0, 1, 3]}, 14: {'values': [5, 10]}, 15: {'values': [6, 10]}, 16: {'values': [1]}, 17: {'values': [0]}, 18: {'values': [7]}, -1: {'values': []}, 19: {'values': [0, 1, 3]}, 20: {'values': [0, 1, 3]}})

In [52]:
pcm_graph.edges

EdgeView([(0, 9), (0, 10), (1, 11), (1, 12), (2, 13), (2, 16), (3, 17), (3, 19), (4, 14), (4, 15), (5, 12), (5, 18), (6, 9), (6, 14), (7, 10), (7, 20), (8, 11), (8, 15), (13, 19), (13, 20), (16, -1)])

In [53]:
handle_join_nodes(pcm_graph,pcm_root,None)

In [54]:
pcm_graph.nodes(data=True)

NodeDataView({0: {'values': [3, 4]}, 1: {'values': [6, 7, 9]}, 2: {'values': [1, 2]}, 3: {'values': [0, 1]}, 4: {'values': [5, 6, 10]}, 5: {'values': [7, 8]}, 6: {'values': [4, 5, 10]}, 7: {'values': [1, 3]}, 8: {'values': [6, 9, 10]}, 9: {'values': [4]}, 10: {'values': [3]}, 11: {'values': [6, 9]}, 12: {'values': [7]}, 13: {'values': [0, 1, 3]}, 14: {'values': [5, 10]}, 15: {'values': [6, 10]}, 16: {'values': [1]}, 17: {'values': [0]}, 18: {'values': [7]}, -1: {'values': []}, 19: {'values': [0, 1, 3]}, 20: {'values': [0, 1, 3]}})

In [55]:
pcm_graph.nodes

NodeView((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, -1, 19, 20))

In [56]:
def handle_amnesiac_node(graph, root, child, root_values_set, child_values_set):
  """ This function handles an "amnesiac node": |B_parent| < |B_child|-1

      I.e. where the parent contains 2 or fewer elements than the child does. This
      indicates that the parent has "forgotten too much" in one step (is amnesiac)
  """

  assert type(root_values_set) == set
  assert type(child_values_set) == set

  # Find the "forgotten values", i.e. those values which are present in the bag
  # of the child but not in the bag of the parent 
  forgotten_values = list(child_values_set-root_values_set)
  assert len(forgotten_values) > 1, f"We expect 2 or more elements forgotten, but only got these: {forgotten_values}"
  # print(f"Forget case. Root elements: {root_values_set}, child elements: {child_values_set}")

  # Constructing a new, bridging node.
  # I will arbitrarily pick the first element of the forgotten nodes
  element_to_forget = forgotten_values[0]

  # The bag of the new, bridging node is constructed. This is simply done by appending
  # the arbitrarily chosen forgotten element to the bag of the parent:
  new_node_bag = root_values_set.copy()
  new_node_bag.add(element_to_forget)

  # id of the new node is just obtained by incrementing the current max in the graph by 1
  new_node_id = max(graph.nodes)+1
  # print("adding new forget node: ", new_node_id)
  # print("With the bag: ", new_node_bag)

  # removing the old edge between root and child
  # inserting the new node between the root and child
  insert_bridging_node(graph,root,child,new_node_id,list(new_node_bag))
  return new_node_id
    

In [57]:

def handle_eager_introducer(graph, root, child, root_values_set, child_values_set):
  """This function handles an eager introducer node: |B_parent| > |B_child|+1

      I.e. where the bag of the parent contains 2 or more elements more than the 
      bag of the child
  """
  
  assert type(root_values_set) == set
  assert type(child_values_set) == set

  # Find the surplus of introduced nodes (i.e. those nodes in the parent bag
  # but not in the child bag)
  introduced_surplus = list(root_values_set-child_values_set)
  element_to_introduce = introduced_surplus[0]

  # The bag of the new bridging node is constructed by removing one element
  # from the bag of the parent node
  new_node_bag = root_values_set.copy()
  new_node_bag.remove(element_to_introduce)
  new_node_id = max(graph.nodes)+1
  # print("adding new introduce node: ", new_node_id)
  # print("With the bag: ",new_node_bag)
  insert_bridging_node(graph, root, child, new_node_id, list(new_node_bag))
  return new_node_id

In [58]:


def handle_identical(graph,root,child,root_values,child_values):
  """ This function handles the case where the bags of the parent and child are identical:
      B_parent = B_child

      This is handled by creating a bridging node which contains the bag of the parent
      with one element removed 
  """
  
  assert type(root_values) == set
  assert type(child_values) == set
  element_to_remove = tuple(root_values)[0]
  new_node_bag = root_values.copy()
  new_node_bag.remove(element_to_remove)
  new_node_id = max(graph.nodes)+1
  insert_bridging_node(graph,root,child,new_node_id,list(new_node_bag))
  return new_node_id

In [59]:
def correct_unnice_node(graph, root, child, is_child_of_join_node):
  # print("EVALUATING root: ",root)
  assert type(child) == int, f"expect int type for child but type is: {type(child)}"

  # print(f"current node {root} has child: {child}")

  root_values = graph.nodes[root]["values"]
  child_values = graph.nodes[child]["values"]
  root_values_set = set(root_values)
  child_values_set = set(child_values)

  if is_child_of_join_node == False and root_values_set == child_values_set:
    ### IDENTICAL CASE
    # print("Root and child sets are identical. Must insert a bridging node with one random element removed.")
    new_node_id = handle_identical(graph,root,child,root_values_set,child_values_set)
    make_decomposition_nice(graph, new_node_id, root)
  elif len(root_values_set-child_values_set) > 0 and len(child_values_set-root_values_set) > 0:
    ### AMBIVALENT CASE
    # print("The root values contain  at least one element not present in child values, and vice versa. This means that the root both forgets and introduces at the same time")
    new_node_id = handle_eager_introducer(graph,root,child,root_values_set,child_values_set)
    make_decomposition_nice(graph, new_node_id, root)
  elif len(root_values_set-child_values_set) > 1:
    ### EAGER INTRODUCER CASE
    # print("Root values contains 2 or more elements more than child value does. Introduces more than 1 element ")
    new_node_id = handle_eager_introducer(graph,root,child,root_values_set,child_values_set)
    make_decomposition_nice(graph, new_node_id, root)
  elif len(child_values_set-root_values_set) > 1:
    ### AMNESIAC PARENT CASE
    # print("Child value contains 2 or more elements more than parent. must forget too much at once")
    new_node_id = handle_amnesiac_node(graph,root,child,root_values_set,child_values_set)
    make_decomposition_nice(graph, new_node_id, root)
  else:
    # print(f"Current node {root} has no violation with child {child}. Recursing on child again")
    make_decomposition_nice(graph,child,root)



# Recursive (well bi-directionally recursive) function that goes through the decomposition and calls to fix nodes
def make_decomposition_nice(graph, root, parent):
  # Find the children of this current root node by removing the parent from
  # the list of its neighbors
  children = list(graph.neighbors(root))
  try:
    children.remove(parent)
  except ValueError:
    print(f"Current root: {root} has no parent, is probably root of entire tree")

  # print(f"In recursion. Root: {root},  children: {children}")
  # If this node has no children, it must be a root
  if len(children) == 0:
    # print(f"len of children of this node: {root} is 0, must be a leaf. returning")
    return
  # Otherwise, if this node has 2 children, it must be a join node
  elif len(children) > 1:
    # print(f"current node {root} has more than 1 child and must be a join node")
    assert len(children) == 2, f"Expected children to be of length 2 but is: {len(children)}"
    # In the case of a join node, we branch out to each of the children
    correct_unnice_node(graph, root, children[0], True)
    correct_unnice_node(graph, root, children[1], True)
  else:
    # Otherwise, the current root has just one child and we continue
    assert len(children) == 1, f"Expected children to be of length 1 but is: {len(children)}"
    correct_unnice_node(graph, root, children[0], False)


In [60]:
# Entry point in a graph given its root
def construct_nice_decomposition(input_graph):
  graph, root = construct_junction_and_add_leaves(input_graph)
  handle_promiscuous_nodes2(graph,root,None)
  handle_join_nodes(graph,root,None)
  # print(f"In entry point. Root: {root}")
  make_decomposition_nice(graph, root, None)
  return graph, root

In [61]:
pcm_graph, pcm_root = construct_junction_and_add_leaves(pcm)
handle_promiscuous_nodes2(pcm_graph,pcm_root,None)
handle_join_nodes(pcm_graph,pcm_root,None)
print(pcm_graph.nodes(data=True))
print(pcm_graph.edges)
# make_decomposition_nice(pcm_graph,pcm_root)
pcm_nice = construct_nice_decomposition(pcm_graph)

[(0, {'values': [3, 4]}), (1, {'values': [6, 7, 9]}), (2, {'values': [1, 2]}), (3, {'values': [0, 1]}), (4, {'values': [5, 6, 10]}), (5, {'values': [7, 8]}), (6, {'values': [4, 5, 10]}), (7, {'values': [1, 3]}), (8, {'values': [6, 9, 10]}), (9, {'values': [4]}), (10, {'values': [3]}), (11, {'values': [6, 9]}), (12, {'values': [7]}), (13, {'values': [0, 1, 3]}), (14, {'values': [5, 10]}), (15, {'values': [6, 10]}), (16, {'values': [1]}), (17, {'values': [0]}), (18, {'values': [7]}), (-1, {'values': []}), (19, {'values': [0, 1, 3]}), (20, {'values': [0, 1, 3]})]
[(0, 9), (0, 10), (1, 11), (1, 12), (2, 13), (2, 16), (3, 17), (3, 19), (4, 14), (4, 15), (5, 12), (5, 18), (6, 9), (6, 14), (7, 10), (7, 20), (8, 11), (8, 15), (13, 19), (13, 20), (16, -1)]
Current root: -1 has no parent, is probably root of entire tree


In [62]:
pcm_graph.nodes(data=True)

NodeDataView({0: {'values': [3, 4]}, 1: {'values': [6, 7, 9]}, 2: {'values': [1, 2]}, 3: {'values': [0, 1]}, 4: {'values': [5, 6, 10]}, 5: {'values': [7, 8]}, 6: {'values': [4, 5, 10]}, 7: {'values': [1, 3]}, 8: {'values': [6, 9, 10]}, 9: {'values': [4]}, 10: {'values': [3]}, 11: {'values': [6, 9]}, 12: {'values': [7]}, 13: {'values': [0, 1, 3]}, 14: {'values': [5, 10]}, 15: {'values': [6, 10]}, 16: {'values': [1]}, 17: {'values': [0]}, 18: {'values': [7]}, -1: {'values': []}, 19: {'values': [0, 1, 3]}, 20: {'values': [0, 1, 3]}})

In [63]:
pcm_graph.edges

EdgeView([(0, 9), (0, 10), (1, 11), (1, 12), (2, 13), (2, 16), (3, 17), (3, 19), (4, 14), (4, 15), (5, 12), (5, 18), (6, 9), (6, 14), (7, 10), (7, 20), (8, 11), (8, 15), (13, 19), (13, 20), (16, -1)])

In [64]:
construct_nice_decomposition(pcm_graph)

Current root: -1 has no parent, is probably root of entire tree


(<networkx.classes.graph.Graph at 0x25a4d323990>, -1)

In [65]:
pcm_graph.nodes(data=True)

NodeDataView({0: {'values': [3, 4]}, 1: {'values': [6, 7, 9]}, 2: {'values': [1, 2]}, 3: {'values': [0, 1]}, 4: {'values': [5, 6, 10]}, 5: {'values': [7, 8]}, 6: {'values': [4, 5, 10]}, 7: {'values': [1, 3]}, 8: {'values': [6, 9, 10]}, 9: {'values': [4]}, 10: {'values': [3]}, 11: {'values': [6, 9]}, 12: {'values': [7]}, 13: {'values': [0, 1, 3]}, 14: {'values': [5, 10]}, 15: {'values': [6, 10]}, 16: {'values': [1]}, 17: {'values': [0]}, 18: {'values': [7]}, -1: {'values': []}, 19: {'values': [0, 1, 3]}, 20: {'values': [0, 1, 3]}})

In [66]:

def count_matchings_simple(graph):
  edge_list = list(graph.edges)
  matchings = 0
  for r in range(1+math.floor(graph.number_of_nodes()/2)):
    r_combinations = itertools.combinations(edge_list,r)
    for comb in r_combinations:
      if (nx.is_matching(graph,comb) == True):
        matchings += 1
  return matchings

In [67]:
pcm_graph, pcm_root = construct_junction_and_add_leaves(pcm)
handle_promiscuous_nodes2(pcm_graph,pcm_root,None)
handle_join_nodes(pcm_graph,pcm_root,None)
print(pcm_graph.nodes)
print(pcm_graph.edges)
construct_nice_decomposition(pcm_graph)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, -1, 19, 20]
[(0, 9), (0, 10), (1, 11), (1, 12), (2, 13), (2, 16), (3, 17), (3, 19), (4, 14), (4, 15), (5, 12), (5, 18), (6, 9), (6, 14), (7, 10), (7, 20), (8, 11), (8, 15), (13, 19), (13, 20), (16, -1)]
Current root: -1 has no parent, is probably root of entire tree


(<networkx.classes.graph.Graph at 0x25a4d3385d0>, -1)

In [68]:
pcm_graph.nodes(data=True)

NodeDataView({0: {'values': [3, 4]}, 1: {'values': [6, 7, 9]}, 2: {'values': [1, 2]}, 3: {'values': [0, 1]}, 4: {'values': [5, 6, 10]}, 5: {'values': [7, 8]}, 6: {'values': [4, 5, 10]}, 7: {'values': [1, 3]}, 8: {'values': [6, 9, 10]}, 9: {'values': [4]}, 10: {'values': [3]}, 11: {'values': [6, 9]}, 12: {'values': [7]}, 13: {'values': [0, 1, 3]}, 14: {'values': [5, 10]}, 15: {'values': [6, 10]}, 16: {'values': [1]}, 17: {'values': [0]}, 18: {'values': [7]}, -1: {'values': []}, 19: {'values': [0, 1, 3]}, 20: {'values': [0, 1, 3]}})

In [69]:
count_matchings_simple(pcm)

140

In [70]:
g = nx.Graph()
g.add_nodes_from([1,2,3,4,5,6,7])
g.add_edges_from([(1,2),(2,3),(3,4),(4,5),(5,6),(5,7)])

In [71]:
def powerset(iterable,min_inclusive,max_exclusive):
    s = list(iterable)
    return itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(min_inclusive, max_exclusive))

# This should not necessarily be commented out, just testing.

In [72]:
# def count_matchings_forcibly_saturate_vertex(input_graph, vertex_to_saturate):
#   neighbors = list(input_graph.neighbors(vertex_to_saturate))
#   # print()
#   # print()
#   # print("IN COUNT FORIBLY SAT")
#   # print("neighbors: ",neighbors)
#   matchings = []
#   for saturation_endpoint in neighbors:
#     graph = input_graph.copy()
#     this_edge = ((vertex_to_saturate,saturation_endpoint),(saturation_endpoint,vertex_to_saturate))

#     other_neighbors = neighbors.copy()
#     other_neighbors.remove(saturation_endpoint)
#     for other_neighbor in other_neighbors:
#       graph.remove_edge(vertex_to_saturate,other_neighbor)

#     # print(f"After removing neighbors from {vertex_to_saturate}, nodes,edges:")
#     # print(graph.nodes)
#     # print(graph.edges)


#     # print("this edge is: ",this_edge)
#     all_other_edges = list(graph.edges())
#     true_edge = None
#     try:
#       all_other_edges.remove(this_edge[0])
#       true_edge = this_edge[0]
#     except:
#       all_other_edges.remove(this_edge[1])
#       true_edge = this_edge[1]

#     powerset_of_other_edges = powerset(all_other_edges,1,round(len(graph.nodes)/2))

#     count = 1
#     for subset_of_other_edges in list(powerset_of_other_edges):
#       # print("These matching? :", true_edge, subset_of_other_edges, nx.is_matching(graph,set({true_edge,*subset_of_other_edges})))
#       if nx.is_matching(graph,set({true_edge, *subset_of_other_edges})):
#         count += 1
#     matchings.append(count)

#   print("Before returning. matchigns is: ",matchings)
#   return sum(matchings)

In [73]:
def can_remove_partner_vertex_neighbors_in_instance(partner_vertex_neighbors_to_remove,saturation_direction_instance):
  for neighbor_to_remove in partner_vertex_neighbors_to_remove:
    if neighbor_to_remove in saturation_direction_instance:
      # print(f"Heyeyey this won't go. We are trying to remove {neighbor_to_remove} but it is part of the saturtion direction instnace: {saturation_direction_instance}")
      return False
  return True

# original dirty

In [74]:
def count_matchings_vertex_included_multiple(input_graph, vertices_to_saturate:list):
  if vertices_to_saturate != list:
    vertices_to_saturate = list(vertices_to_saturate)
  assert type(vertices_to_saturate) == list
  vertices_to_consider = copy.deepcopy(vertices_to_saturate)
  # print("Vertices to consider after init: ",vertices_to_consider)
  assert vertices_to_consider == vertices_to_saturate
  # print("Vertices to consider1",vertices_to_consider)

  # vertices_to_saturate_which_are_neighbors = []
  # # print("Vertices to consider2",vertices_to_consider)
  # for i in range(len(vertices_to_saturate)):
  #   # print("Vertices to consider just after i iterator",vertices_to_consider)
  #   for j in range(i+1,len(vertices_to_saturate)):
  #     # print("Vertices to consider just after j iterator",vertices_to_consider)
  #     v1 = vertices_to_saturate[i]
  #     v2 = vertices_to_saturate[j]
  #     if input_graph.has_edge(v1,v2):
  #       # print(f"These are adjacent: {v1}, {v2}")
  #       vertices_to_saturate_which_are_neighbors.append((v1,v2))
  #       try:
  #         vertices_to_consider.remove(v1)
  #         vertices_to_consider.remove(v2)
  #       except:
  #         # TODO: Verify the veracity of this. If we are not able to remove, it must mean that we have already removed.
  #         # This example can occur if we wish to saturate the vertices {1,2,3} in the graph but that we have edges
  #         # (1,2), (2,3).
  #         return 0
  #       # except:
  #       #   print(f"exception while removing v  {v1} from the list: {vertices_to_consider}")
  #       #   print("input graph nodes, edges:")
  #       #   print(input_graph.nodes)
  #       #   print(input_graph.edges)
  #       # try:
  #       #   vertices_to_consider.remove(v2)
  #       # except:
  #       #   print(f"exception while removing v2 {v2} from the list: {vertices_to_consider}")
  # print(f"Vertices to saturate which are neighbors: {vertices_to_saturate_which_are_neighbors}")
  # List of lists. [[v1.neighbors],[v2.neighbors],[..],...]
  # Cartesian product of v1.neighbors, v2.neighbors, ...
  # So each entry in this variable representss one way of saturating the vertices
  # which must be saturated. Thereby they are fixed, and we then count no. of matchings
  # in the subgraph that results
  vertices_to_saturate_neighbors = [list(input_graph.neighbors(x)) for x in vertices_to_consider]
  # Ways that these vertices to saturate may be saturated
  saturation_possibilities = list(itertools.product(*vertices_to_saturate_neighbors))
  # print("sat possibilities: ", saturation_possibilities)


  total_matchings = 0
  for saturation_direction_instance in saturation_possibilities:
    graph = input_graph.copy()
    # print(f"After graph copy, nodes: {graph.nodes}, edges: {graph.edges}")
    # if len(vertices_to_saturate_which_are_neighbors) > 0:
    #   for pair in vertices_to_saturate_which_are_neighbors:
    #     for vertex in pair:
    #       for neighbor in list(graph.neighbors(vertex)):
    #         if neighbor not in pair:
    #           print(f"Removing edge between {vertex} and {neighbor}")
    #           try:
    #             graph.remove_edge(vertex,neighbor)
    #           except:
    #             print("Except")
    #             print("Except")
    #             print("Except")
    #             print("Except")
    # if len(saturation_possibilities) == 0:
    #   print("We have no saturation possibilities.")

    neighbor_vertices_taken_up_for_matching = dict()
    # print(f"Evaluating instance: {saturation_direction_instance}")
    if len(saturation_direction_instance) > len(set(saturation_direction_instance)):
      # print("There is a duplicate saturation direction vertex. This must be impossible. Continuing to next iteration")
      continue
    assert len(saturation_direction_instance) == len(vertices_to_consider)
    saturation_direction_instance_is_valid = True
    for i in range(len(saturation_direction_instance)):
      # "home vertex" is the vertex of vertices_to_consider that is currently
      # under investigation. its partner is the neighbor to which it is
      # matched in the current instance.
      if saturation_direction_instance_is_valid == False:
        # print("Breaking, saturation direction instance is invalid")
        break
      home_vertex = vertices_to_consider[i]
      partner_vertex = saturation_direction_instance[i]
      # print(f"Home vertex: {home_vertex}, partner_vertex: {partner_vertex}")
      if graph.has_edge(home_vertex, partner_vertex) == False:
        # print(f"ERROR: graph currently has no edge between home vertex {home_vertex} and partner {partner_vertex}, edges: {graph.edges}")
        saturation_direction_instance_is_valid = False
        continue
      partner_vertex_neighbors_to_remove = list(graph.neighbors(partner_vertex))
      partner_vertex_neighbors_to_remove.remove(home_vertex)
      # print("Partner vertex neighbors to remove: ",partner_vertex_neighbors_to_remove)
      # if can_remove_partner_vertex_neighbors_in_instance(partner_vertex_neighbors_to_remove,saturation_direction_instance) == False:
      #   saturation_direction_instance_is_valid = False
      #   continue
      # for neighbor_to_remove in partner_vertex_neighbors_to_remove:
      #   if neighbor_to_remove in saturation_direction_instance:
      #     print(f"Heyeyey this won't go. We are trying to remove {neighbor_to_remove} but it is part of the saturtion direction instnace: {saturation_direction_instance}")
      #     break
      # If the designated partner of this home vertex is already taken up for matching by another
      # vertex, the current saturation direction instance cannot fulfill the constraints, and we must return 0
      if partner_vertex in neighbor_vertices_taken_up_for_matching:
        # print(f"Ey problem, the designated partner {partner_vertex} of the home vertex {home_vertex} is already taken up by the home vertex {neighbor_vertices_taken_up_for_matching[partner_vertex]}")
        # (return 0)
        # TODO: Fix this, should it return 0 globally?
        raise Exception("eyeyeyeyeyey")
        return 0
      else:
         neighbor_vertices_taken_up_for_matching[partner_vertex] = home_vertex
      # Iterate over neighbors of the home vertex and remove all edges from
      # home vertex to its neighbors (except its designated partner)
      # print("Home vertex is: ",home_vertex)
      home_vertex_neighbors = list(graph.neighbors(home_vertex))
      # print(f"Attempting to remove {partner_vertex} from {home_vertex_neighbors}")
      home_vertex_neighbors.remove(partner_vertex)
      for home_neighbor_to_unedge in home_vertex_neighbors:
        # print(f"Removing edge between {home_vertex} and {home_neighbor_to_unedge}")
        graph.remove_edge(home_vertex,home_neighbor_to_unedge)
      # for neighbor_of_home_vertex in list(graph.neighbors(home_vertex)):
      #   if 
      for edge_to_remove in partner_vertex_neighbors_to_remove:
        # print(f"Removing edge between {partner_vertex} and {edge_to_remove}")
        graph.remove_edge(partner_vertex,edge_to_remove)
    if saturation_direction_instance_is_valid == False:
      continue
    edges_except_constraints = list(graph.edges)
    # print("EEC immediately after init: ", edges_except_constraints)
    # This is quite ghetto but is just to avoid creating a copy of the entire graph for the sole purpose of removing
    # these edges.
    # for i in range(len(vertices_to_consider)):
    #   print(f"Trying to remove: {vertices_to_consider[i],saturation_direction_instance[i]} from {edges_except_constraints}")
    #   if len(edges_except_constraints) > 0:
    #     try:
    #       edges_except_constraints.remove((vertices_to_consider[i],saturation_direction_instance[i]))
    #     except:
    #       edges_except_constraints.remove((saturation_direction_instance[i],vertices_to_consider[i]))
    # for pair in vertices_to_saturate_which_are_neighbors:
    #   try:
    #     edges_except_constraints.remove(pair)
    #   except ValueError:
    #     print(f"pair not in list: ", pair)
    #     print("the list: ", edges_except_constraints)
    #     edges_except_constraints.remove((pair[1],pair[0]))


    ps = powerset(edges_except_constraints,1,round(len(graph.nodes)/2))
    # print(f"About to construct saturation edges in instance. Sat direction instance: {saturation_direction_instance} vertices to consider: {vertices_to_consider}")
    # print("saturation edges in instance:")
    saturation_edges_in_instance = set()
    for i in range((len(vertices_to_consider))):
      saturation_edges_in_instance.add(tuple(sorted((saturation_direction_instance[i],vertices_to_consider[i]))))
    # saturation_edges_in_instance = [(saturation_direction_instance[i],vertices_to_consider[i]) for i in range(len(vertices_to_consider))]
    # print(saturation_edges_in_instance)
    # print(f"Setting local matchings to 1 for these edges: {saturation_edges_in_instance}")
    # print(f"Setting local matchings to 1 for these edges: {saturation_edges_in_instance}")
    # print(f"Setting local matchings to 1 for these edges: {saturation_edges_in_instance}")
    local_matchings = 1
    for element_in_powerset in ps:
      edges_to_evaluate = [*saturation_edges_in_instance,*element_in_powerset]
      # print(f"Evaluating is matching?: {edges_to_evaluate}")
      is_matching = nx.is_matching(graph, edges_to_evaluate)
      if is_matching:
        # print(f"This is matching: {edges_to_evaluate} in the graph w edges {graph.edges}")
        # print(f"This is matching: {edges_to_evaluate} in the graph w edges {graph.edges}")
        # print(f"This is matching: {edges_to_evaluate} in the graph w edges {graph.edges}")
        local_matchings += 1
    total_matchings += local_matchings
  #   print("LOCAL MATCHINGS COUNTER: ",local_matchings)
  #   print("TOTAL MATCHINGS COUNTER: ",total_matchings)
  # print("Returning: ",total_matchings)
  return total_matchings

# New version. <h1 style="color:red;"> I BROKE THIS ! </h1>

In [75]:
def count_matchings_vertex_included_multiple(input_graph, vertices_to_saturate:list):
  if vertices_to_saturate != list:
    vertices_to_saturate = list(vertices_to_saturate)
  assert type(vertices_to_saturate) == list
  vertices_to_consider = copy.deepcopy(vertices_to_saturate)
  # print("Vertices to consider after init: ",vertices_to_consider)
  assert vertices_to_consider == vertices_to_saturate
  # print("Vertices to consider1",vertices_to_consider)

  # vertices_to_saturate_which_are_neighbors = []
  # # print("Vertices to consider2",vertices_to_consider)
  # for i in range(len(vertices_to_saturate)):
  #   # print("Vertices to consider just after i iterator",vertices_to_consider)
  #   for j in range(i+1,len(vertices_to_saturate)):
  #     # print("Vertices to consider just after j iterator",vertices_to_consider)
  #     v1 = vertices_to_saturate[i]
  #     v2 = vertices_to_saturate[j]
  #     if input_graph.has_edge(v1,v2):
  #       # print(f"These are adjacent: {v1}, {v2}")
  #       vertices_to_saturate_which_are_neighbors.append((v1,v2))
  #       try:
  #         vertices_to_consider.remove(v1)
  #         vertices_to_consider.remove(v2)
  #       except:
  #         # TODO: Verify the veracity of this. If we are not able to remove, it must mean that we have already removed.
  #         # This example can occur if we wish to saturate the vertices {1,2,3} in the graph but that we have edges
  #         # (1,2), (2,3).
  #         return 0
  #       # except:
  #       #   print(f"exception while removing v  {v1} from the list: {vertices_to_consider}")
  #       #   print("input graph nodes, edges:")
  #       #   print(input_graph.nodes)
  #       #   print(input_graph.edges)
  #       # try:
  #       #   vertices_to_consider.remove(v2)
  #       # except:
  #       #   print(f"exception while removing v2 {v2} from the list: {vertices_to_consider}")
  # print(f"Vertices to saturate which are neighbors: {vertices_to_saturate_which_are_neighbors}")
  # List of lists. [[v1.neighbors],[v2.neighbors],[..],...]
  # Cartesian product of v1.neighbors, v2.neighbors, ...
  # So each entry in this variable representss one way of saturating the vertices
  # which must be saturated. Thereby they are fixed, and we then count no. of matchings
  # in the subgraph that results
  vertices_to_saturate_neighbors = [list(input_graph.neighbors(x)) for x in vertices_to_consider]
  # Ways that these vertices to saturate may be saturated
  saturation_possibilities = list(itertools.product(*vertices_to_saturate_neighbors))
  # print("sat possibilities: ", saturation_possibilities)
  total_matchings = 0

  for saturation_direction_instance in saturation_possibilities:
    graph = input_graph.copy()


    neighbor_vertices_taken_up_for_matching = dict()
    # print(f"Evaluating instance: {saturation_direction_instance}")
    if len(saturation_direction_instance) > len(set(saturation_direction_instance)):
      # print("There is a duplicate saturation direction vertex. This must be impossible. Continuing to next iteration")
      continue
    assert len(saturation_direction_instance) == len(vertices_to_consider)
    saturation_direction_instance_is_valid = True
    for i in range(len(saturation_direction_instance)):
      # "home vertex" is the vertex of vertices_to_consider that is currently
      # under investigation. its partner is the neighbor to which it is
      # matched in the current instance.
      if saturation_direction_instance_is_valid == False:
        # print("Breaking, saturation direction instance is invalid")
        break
      home_vertex = vertices_to_consider[i]
      partner_vertex = saturation_direction_instance[i]
      # print(f"Home vertex: {home_vertex}, partner_vertex: {partner_vertex}")
      if graph.has_edge(home_vertex, partner_vertex) == False:
        # print(f"ERROR: graph currently has no edge between home vertex {home_vertex} and partner {partner_vertex}, edges: {graph.edges}")
        saturation_direction_instance_is_valid = False
        continue
      partner_vertex_neighbors_to_remove = list(graph.neighbors(partner_vertex))
      partner_vertex_neighbors_to_remove.remove(home_vertex)
      # print("Partner vertex neighbors to remove: ",partner_vertex_neighbors_to_remove)
    
      # If the designated partner of this home vertex is already taken up for matching by another
      # vertex, the current saturation direction instance cannot fulfill the constraints, and we must return 0
      if partner_vertex in neighbor_vertices_taken_up_for_matching:
        # print(f"Ey problem, the designated partner {partner_vertex} of the home vertex {home_vertex} is already taken up by the home vertex {neighbor_vertices_taken_up_for_matching[partner_vertex]}")
        # (return 0)
        # TODO: Fix this, should it return 0 globally?
        raise Exception("eyeyeyeyeyey")
        return 0
      else:
         neighbor_vertices_taken_up_for_matching[partner_vertex] = home_vertex
      # Iterate over neighbors of the home vertex and remove all edges from
      # home vertex to its neighbors (except its designated partner)
      home_vertex_neighbors = list(graph.neighbors(home_vertex))
      home_vertex_neighbors.remove(partner_vertex)
      two_for_loops_start = time.time()
      for home_neighbor_to_unedge in home_vertex_neighbors:
        graph.remove_edge(home_vertex,home_neighbor_to_unedge)
      for edge_to_remove in partner_vertex_neighbors_to_remove:
        graph.remove_edge(partner_vertex,edge_to_remove)
    if saturation_direction_instance_is_valid == False:
      continue

    powerset_start = time.time()

    edges_except_constraints = list(graph.edges)
    ps = powerset(edges_except_constraints,1,round(len(graph.nodes)/2))
    saturation_edges_in_instance = set()
    for i in range((len(vertices_to_consider))):
      saturation_edges_in_instance.add(tuple(sorted((saturation_direction_instance[i],vertices_to_consider[i]))))

    powerset_end = time.time()


    xy = 0
    matching_time = 0
    local_matchings = 1

    after_ps_start = time.time()
    print(f"powerset length is: {len(list(ps))}")
    for element_in_powerset in ps:
      x = time.time()
      edges_to_evaluate = [*saturation_edges_in_instance,*element_in_powerset]
      y = time.time()
      xy += (y-x)
      t1 = time.time()
      is_matching = nx.is_matching(graph, edges_to_evaluate)
      t2 = time.time()
      matching_time += (t2-t1)
      if is_matching:
        local_matchings += 1
    after_ps_end = time.time()

    total_matchings += local_matchings
    print()
    print(f"FOR THIS SATURATION DIRECTION INSTANCE {saturation_direction_instance}")
    print(f"make power set, add stuff to saturation edges in instance etc: {powerset_end-powerset_start}")
    print(f"stuff after power set: {after_ps_end-after_ps_start}")
    print(f"Time spent using is_matching: {matching_time}")
    print(f"Time spent creating edges_to_evaluate: {xy}")
  return total_matchings

In [76]:
count_matchings_vertex_included_multiple(g,[1,2,6])

powerset length is: 7

FOR THIS SATURATION DIRECTION INSTANCE (2, 1, 5)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0


1

In [77]:
count_matchings_vertex_included_multiple(g,[1,3,5,6])

powerset length is: 7

FOR THIS SATURATION DIRECTION INSTANCE (2, 4, 6, 5)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0


1

In [78]:
def count_matchings_forcibly_saturate_and_unsaturate(input_graph, vertices_to_saturate: list, vertices_to_unsaturate: list):
    graph = input_graph.copy()
    if type(vertices_to_unsaturate) != list:
        vertices_to_unsaturate = list(vertices_to_unsaturate)
    if len(vertices_to_unsaturate) > 0:
        graph.remove_nodes_from(vertices_to_unsaturate)
    # print("Calling count matchings vult")
    return count_matchings_vertex_included_multiple(graph,vertices_to_saturate)

In [79]:
count_matchings_forcibly_saturate_and_unsaturate(g,[2],[6])

powerset length is: 10

FOR THIS SATURATION DIRECTION INSTANCE (1,)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0012998580932617188
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 6

FOR THIS SATURATION DIRECTION INSTANCE (3,)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0


2

In [80]:
# state_results = {}

# def count_matchings_in_nice_tree_decomp(graph, decomposition, current, parent):
#   children = list(decomposition.neighbors(current))
#   try:
#     children.remove(parent)
#   except ValueError:
#     print("Current element has no parent, must be root")
#   while len(children) > 0:
#     if len(children) == 2:
#       assert len(children) == 2, f"Expected 2 children, got {len(children)}"
#       assert type(children) == list, f"TYpe of children should be list, it is {children}"
#       sum_left, vertices_encountered_left = count_matchings_in_nice_tree_decomp(graph, decomposition,children[0],current)
#       sum_right, vertices_encountered_right = count_matchings_in_nice_tree_decomp(graph, decomposition,children[1],current)
#       if vertices_encountered_left.issubset(vertices_encountered_right) or vertices_encountered_right.issubset(vertices_encountered_left):
#         print("Ey whats going on two subgraphs are identical?")
#         return max(sum_left,sum_right), vertices_encountered_left.union(vertices_encountered_right)  
#       else:    
#         print("At join node. Current: ",current)
#         print("VE_left: ",vertices_encountered_left)
#         print("VE_right: ",vertices_encountered_right)
#         induced_subgraph_left = graph.subgraph(vertices_encountered_left)
#         induced_subgraph_right = graph.subgraph(vertices_encountered_right)
#         current_bag = decomposition.nodes[current]["values"]
#         powerset_of_current_bag = list(powerset(current_bag,0,len(current_bag)+1))
#         print(f"About to branch out left and right. current_bag: {current_bag}, induced subgraph nodes left, right: {induced_subgraph_left.nodes} and {induced_subgraph_right.nodes}")
#         running_sum = 0
#         for subset_of_bag in powerset_of_current_bag:
#           print(f"subset of bag: {subset_of_bag}")
#           left_sum = count_matchings_forcibly_saturate_and_unsaturate(induced_subgraph_left,list(subset_of_bag),set(current_bag).difference(set(subset_of_bag)))
#           right_sum = count_matchings_forcibly_saturate_and_unsaturate(induced_subgraph_right,set(current_bag).difference(set(subset_of_bag)),list(subset_of_bag))
#           running_sum += left_sum * right_sum

#         # print("After two recursive calls at join node: ",current)
#         vertices_encountered = vertices_encountered_left.union(vertices_encountered_right)
#         # vertices_encountered.update(set(decomposition.nodes[current]["values"]))
        
#         return sum_left+sum_right+ running_sum, vertices_encountered
    
#     elif len(children) == 1:
#       assert len(children) == 1, f"Expected 1 child, got {len(children)}"
#       print("One child case. Current: ",current)
#       local_sum, vertices_encountered = count_matchings_in_nice_tree_decomp(graph, decomposition, children[0],current)
#       # print("After recursive call at single node: ", current)

#       child_values = decomposition.nodes[children[0]]["values"]
#       current_values = decomposition.nodes[current]["values"]
#       print(f"Child values: {child_values}")
#       print(f"Current values: {current_values}")
#       if len(child_values) > len(current_values):
#         print("We must be at a forget node. Just passing on the previously fuond value")
#         return local_sum, vertices_encountered
#       else:
#         print("we must be at introduce node")
#         introduced_element = list(set(current_values) - set(child_values))
#         assert len(introduced_element) == 1, f"introduced element should be 1 element but is: {introduced_element}"
#         introduced_element = introduced_element[0]
#         vertices_encountered.update(set(decomposition.nodes[current]["values"]))
#         # vertices_encountered.add(*decomposition.nodes[current]["values"])
#         # print(f"Inducing subgraph for these vertices: {vertices_encountered}")
#         induced_subgraph = graph.subgraph(vertices_encountered)
#         print(f"Induced subgraph nodes: {induced_subgraph.nodes}")
#         matchings_in_induced_subgraph = count_matchings_simple(induced_subgraph)
#         print(f"Matchings in induced subgraph at current node {current} is = {matchings_in_induced_subgraph}")

#         # count matchings in subgraph induced by nodes in vertices_encountered including current

#         new_result = count_matchings_forcibly_saturate_vertex(induced_subgraph,introduced_element)
#         print(f"count matchings included vertex: {current} result: {new_result} ")
#         combined_result = new_result+local_sum
#         print("COMBINED RESULT: ",combined_result)
#         return combined_result, vertices_encountered

#     else:
#       raise Exception("Ey, too many children: ", len(children))
#   # After while loop, we are now at a leaf
#   print("THink we are at leaf, current: ",current)
#   leaf_value = decomposition.nodes[current]["values"]
#   # print(f"Leaf value: {leaf_value}")
#   # vertices_encountered.update(set(leaf_value))
#   # print("after adding initial to v encountered: ", vertices_encountered)
#   # for state in possible_states:
#   #   computation = 1+1+1
#   #   state_results[state] = computation
#   # return vertices_encountered
#   return 1, set(leaf_value)
    

In [81]:
pcm_decomp, pcm_decomp_root = construct_nice_decomposition(pcm)

Current root: -1 has no parent, is probably root of entire tree


In [82]:
# count_matchings_in_nice_tree_decomp(pcm, pcm_decomp,pcm_decomp_root,None)

In [83]:
count_matchings_simple(pcm)

140

In [84]:
construct_nice_decomposition(pcm)

Current root: -1 has no parent, is probably root of entire tree


(<networkx.classes.graph.Graph at 0x25a4d387090>, -1)

In [85]:
halicin = read_smiles("C1=C(SC(=N1)SC2=NN=C(S2)N)[N+](=O)[O-]")
# halicin_graph, halicin_root = construct_junction_and_add_leaves(halicin)
# handle_promiscuous_nodes2(halicin_graph,halicin_root,None)
# handle_join_nodes(halicin_graph,halicin_root,None)
# print(halicin_graph.nodes(data=True))
# print(halicin_graph.edges)
# ENTRY(halicin_graph,halicin_root)

In [86]:
# count_matchings_simple(halicin)

In [87]:
# count_matchings_in_nice_tree_decomp(halicin, halicin_graph,halicin_root,None)

In [88]:
# def binary_strings_are_compatible(string_one,string_two,elements_one,elements_two):
    

# Original, dirty version (dirty w regards to print statements and commented-out lines)

In [89]:
# state_results = {}

# def count_matchings_in_nice_tree_decomp_REMEMBER_STATE(graph, decomposition, current, parent):
#   children = list(decomposition.neighbors(current))
#   try:
#     children.remove(parent)
#   except ValueError:
#     print("Current element has no parent, must be root")
#   print(f"Current node: {current} has these children after removal of parent: {children}")
#   while len(children) > 0:
#     if len(children) == 2:
#       assert len(children) == 2, f"Expected 2 children, got {len(children)}"
#       assert type(children) == list, f"TYpe of children should be list, it is {children}"
#       print("calling left")
#       child_states_left, vertices_encountered_left_set = count_matchings_in_nice_tree_decomp_REMEMBER_STATE(graph, decomposition,children[0],current)
#       print("calling right")
#       child_states_right, vertices_encountered_right_set = count_matchings_in_nice_tree_decomp_REMEMBER_STATE(graph, decomposition,children[1],current)
#       print("AT JOIN NODE")
#       print("AT JOIN NODE")
#       print("AT JOIN NODE")
#       print(f"child states left: ",child_states_left)
#       print(f"child states right: ",child_states_right)
#       if vertices_encountered_left_set.issubset(vertices_encountered_right_set) or vertices_encountered_right_set.issubset(vertices_encountered_left_set):
#         print(f"Ey whats going on, one subgraph is a susbet of the other? {vertices_encountered_left_set} and {vertices_encountered_right_set}")
#         if len(vertices_encountered_left_set) > len(vertices_encountered_right_set):
#           return child_states_left, vertices_encountered_left_set
#         else:
#           return child_states_right, vertices_encountered_right_set
#         # return max(sum_left,sum_right), vertices_encountered_left_set.union(vertices_encountered_right_set)  
#       # else:    
#       print("At join node. Current: ",current)
#       vertices_encountered_left = list(vertices_encountered_left_set)
#       vertices_encountered_right = list(vertices_encountered_right_set)
#       # print("VE_left: ",vertices_encountered_left)
#       # print("VE_right: ",vertices_encountered_right)
#       vertices_encountered_left.sort()
#       vertices_encountered_right.sort()
#       # induced_subgraph_left = graph.subgraph(vertices_encountered_left)
#       # induced_subgraph_right = graph.subgraph(vertices_encountered_right)
#       current_bag = decomposition.nodes[current]["values"]
#       left_child_bag = decomposition.nodes[children[0]]["values"]
#       right_child_bag = decomposition.nodes[children[1]]["values"]
#       current_bag.sort()
#       left_child_bag.sort()
#       right_child_bag.sort()
#       indices_of_current_bag_elements_in_left_bag = [left_child_bag.index(x) for x in current_bag]
#       indices_of_current_bag_elements_in_right_bag = [right_child_bag.index(x) for x in current_bag]
#       powerset_of_current_bag = list(powerset(current_bag,0,len(current_bag)+1))
#       # print(f"About to branch out left and right. current_bag: {current_bag}, induced subgraph nodes left, right: {induced_subgraph_left.nodes} and {induced_subgraph_right.nodes}")
#       # running_sum = 0


#       # outer_sum
#       new_states = {}
#       state_sum_accumulator = 0
#       # Iterates over all possible states at the join node, e.g. "000", "001", "010", "011", etc
#       for state_string_tuple in list(itertools.product("01",repeat=len(current_bag))):
#         state_string = "".join(state_string_tuple)
#         print()
#         print("STATE STRING: ",state_string)
#         # Find the indices in the string which contain a 1
#         # e.g. "010" -> [1],          "011" -> [1,2]
#         indices_of_ones = []
#         for i in range(len(state_string)):
#             if state_string[i] == "1":
#                 indices_of_ones.append(i)
#         # indices_of_ones = [state_string[int(x] for x in state_string if x == "1"]
#         # print("INDIECS OF ONS: ",indices_of_ones)
#         # Compute ways to distribute these ones to the two subgraphs,
#         # e.g. [1] -> (([1],),(,[1])     [1,2] -> (([1,2],),([1],[2]),([2],[1]),(,[1,2]))
#         # state_result = 0
#         # running_sum_left = 0
#         # running_sum_right = 0
#         state_inner_accumulator = 0
#         for indices_of_ones_assigned_to_left in list(powerset(indices_of_ones,0,len(state_string)+1)):
#             indices_of_ones_assigned_to_right = list(set(indices_of_ones)-set(indices_of_ones_assigned_to_left))
#             # print(f"indices assigned left: {indices_of_ones_assigned_to_left} and right: {indices_of_ones_assigned_to_right}")
#             # Now, for e.g. the state string "011" and the index example [1,2] -> ([1],[2]) 
#             # this can be interpreted as "match index 1 of the state string in the left subgraph, match index 2 of the state string in the right subgraph"
#             # I must now fill the "rest" of the state string indices with 0's in order to forcibly unsaturate them
#             # then, I will obtain these signatures: subgraph1: "010" subgraph2: "001"
#             print(f"indiced of ones assigned to left: {indices_of_ones_assigned_to_left} and right: {indices_of_ones_assigned_to_right}")
#             state_string_left = ["0" for x in range(len(state_string))]
#             for index_of_one_to_left in indices_of_ones_assigned_to_left:
#                 state_string_left[int(index_of_one_to_left)] = "1"
                
#             state_string_right = ["0" for x in range(len(state_string))]
#             # print(f"state string right: {state_string_right}")
#             for index_of_one_to_right in indices_of_ones_assigned_to_right:
#                 state_string_right[int(index_of_one_to_right)] = "1"
#             # state_string_left should in the example now contain "010" and the right variable contain "001"
#             print(f"statestringleft: {state_string_left} and statesright: {state_string_right}")

#             # graph_string_left = ["0" for x in range(len(v_enc_left))]
#             # graph_string_right = ["0" for x in range(len(v_enc_right))]

#             # indices_of_ones_in_left_graph = []
#             # indices_of_ones_in_right_graph = []
#             # for i in range(len(state_string_left)):
#             #     if state_string_left[i] == "1":
#             #         indices_of_ones_in_left_graph.append(vertices_encountered_left.index(current_bag[i]))
#             #         # graph_string_left[v_enc_left.index(join_node_bag[i])] = "1"
#             # for i in range(len(state_string_right)):
#             #     if state_string_right[i] == "1":
#             #         indices_of_ones_in_right_graph.append(vertices_encountered_right.index(current_bag[i]))
#             #         # graph_string_right[v_enc_right.index(current_bag[i])] = "1"
#             print(f"Now ready to go, I think. join bag: {current_bag} bags left, right: {vertices_encountered_left} and {vertices_encountered_right}")
#             # print(f"And graph string left right: {graph_string_left} and {graph_string_right}")
#             print()
#             print(f"state string left graph: {state_string_left}")
#             print(f"state string right graph: {state_string_right}")
#             print()


#             left_value = child_states_left["".join(state_string_left)]
#             right_value = child_states_right["".join(state_string_right)]
#             combined_value = left_value*right_value
#             state_inner_accumulator += combined_value
#             print("left value: ",left_value)
#             print("right value: ",right_value)



#             # # Now iterate over all of the stored states from each child. Find those which have the same matching signature
#             # # as the state that we are investigating now. If 
#             # inner_sum_left = 0
#             # for left_child in child_states_left:
#             #   print("left child is: ",left_child)
#             #   print(f"indices of one in left graph: {state_string_left}")
#             #   for i in range(len(state_string_left)):
#             #     no_violation = True
#             #     if left_child[int(state_string_left[i])] != "1":
#             #       no_violation = False
#             #   if no_violation:
#             #     # running_sum_left += child_states_left[left_child[indices_of_ones_in_left_graph[i]]]
#             #     print(f"This left  child fulfills: {left_child}")
#             #     print(child_states_left[left_child])
#             #     inner_sum_left += child_states_left[left_child]
            
#             # inner_sum_right= 0
#             # for right_child in child_states_right:
#             #   print("right child is: ",right_child)
#             #   print(f"indices of one in right graph: {state_string_right}")
#             #   for i in range(len(state_string_right)):
#             #     print("i: ",i)
#             #     no_violation = True
#             #     if right_child[int(state_string_right[i])] != "1":
#             #       no_violation = False
#             #   if no_violation:
#             #     inner_sum_right += child_states_right[right_child]
#             #     # running_sum_right += child_states_right[right_child[indices_of_ones_in_right_graph[i]]]
              
              
#             #   inner_sum_combined = inner_sum_left*inner_sum_right

#               #  no_violation = True
#               #  iteration = 0
#               #  while no_violation:
#               #     print(" in while loop")
#               #     if left_child[indices_of_ones_in_left_graph[iteration]] == "1":
#               #        running_sum_left += left_child[indices_of_ones_in_left_graph[iteration]]
#               #        iteration += 1
#               #     else:
#               #        no_violation = False
#             # for right_child in child_states_right:
#             #    no_violation = True
#             #    iteration = 0
#             #    while no_violation:
#             #       if right_child[indices_of_ones_in_right_graph[iteration]] == "1":
#             #          running_sum_right += right_child[indices_of_ones_in_right_graph[iteration]]
#             #          iteration +=1
#             #       else:
#             #          no_violation = False
#         print(f"")
#         print(f"For the state {state_string}, state inner accumulator= {state_inner_accumulator}")
#         # state_sum_accumulator += combined_value
#         # print(f"For the state {state_string}, the result is runing sum left: {running_sum_left} multiplyed by running irght {running_sum_right} = {running_sum_left*running_sum_right}")
#         new_states[state_string] = state_inner_accumulator
#         # left_vertices_to_saturate = []
#         # left_vertices_to_unsaturate = []
#         # for left_identity_indices in list(powerset(range(len(state_string)),0,len(state_string)+1)):
#         #   right_identity_indices = [x for x in state_string if x not in left_identity_indices]
#         #   for identity_index in left_identity_indices:
#         #     right_identity_indices.remove(identity_index)
#         #     if identity_index == "0":
#         #       left_vertices_to_unsaturate.append(state_string[identity_index])
#         #     else:
#         #       left_vertices_to_saturate.append(state_string[identity_index])

#         # for child_state_left in child_states_left:
#         #   for child_state_right in child_states_right:

#         # child_states_left[state_string] * child_states_right[state_string]
                      

#       # for subset_of_bag in powerset_of_current_bag:
#       #   print(f"subset of bag: {subset_of_bag}")
#       #   left_sum = count_matchings_forcibly_saturate_and_unsaturate(induced_subgraph_left,list(subset_of_bag),set(current_bag).difference(set(subset_of_bag)))
#       #   right_sum = count_matchings_forcibly_saturate_and_unsaturate(induced_subgraph_right,set(current_bag).difference(set(subset_of_bag)),list(subset_of_bag))
#       #   running_sum += left_sum * right_sum

#       # print("After two recursive calls at join node: ",current)
#       vertices_encountered = vertices_encountered_left_set.union(vertices_encountered_right_set)
#       # vertices_encountered.update(set(decomposition.nodes[current]["values"]))
#       print(f"Returning from introduce node {current}. New states: {new_states}")
#       return new_states, vertices_encountered
  
#     elif len(children) == 1:
#       assert len(children) == 1, f"Expected 1 child, got {len(children)}"
#       child_states, vertices_encountered = count_matchings_in_nice_tree_decomp_REMEMBER_STATE(graph, decomposition, children[0],current)
#       print(f"One child case. Current: {current} and child_states: {child_states}")
#       # print("After recursive call at single node: ", current)

#       child_bag = decomposition.nodes[children[0]]["values"]
#       current_bag = decomposition.nodes[current]["values"]
#       child_bag.sort()
#       current_bag.sort()
#       print(f"Child values: {child_bag}")
#       print(f"Current values: {current_bag}")
#       if len(child_bag) > len(current_bag):
#         print(f"We must be at a forget node. aggregating. Current node: {current} and child: {children[0]} ")
#         forgotten_element = list(set(child_bag) - set(current_bag))
#         assert len(forgotten_element) == 1, f"Forgotten element set should only contain one value, but it is: {forgotten_element}"
#         forgotten_element = forgotten_element[0]
#         index_of_forget_element = child_bag.index(forgotten_element)
#         new_states = {}
#         for state_string in child_states:
#           print("In forget node looping over child states. Current child state string: ",state_string)
#           new_string = state_string[:index_of_forget_element] + state_string[index_of_forget_element+1:]
#           try:
#             new_states[new_string] = new_states[new_string] + child_states[state_string]
#           except KeyError:
#              new_states[new_string] = child_states[state_string]


#         print("Returning these new states from forget node: ", new_states)
#         return new_states, vertices_encountered
#       else:
#         print("we must be at introduce node")
#         introduced_element = list(set(current_bag) - set(child_bag))
#         assert len(introduced_element) == 1, f"introduced element should be 1 element but is: {introduced_element}"
#         introduced_element = introduced_element[0]
#         index_of_introduced_element = current_bag.index(introduced_element)
#         print(f"Index of the element {introduced_element} within current values {current_bag} is {index_of_introduced_element}")
#         vertices_encountered.update(set(decomposition.nodes[current]["values"]))
#         # vertices_encountered.add(*decomposition.nodes[current]["values"])
#         # print(f"Inducing subgraph for these vertices: {vertices_encountered}")
#         induced_subgraph = graph.subgraph(vertices_encountered)
#         print(f"Induced subgraph nodes: {induced_subgraph.nodes}")
#         print(f"Induced subgraph edges: {induced_subgraph.edges}")
#         # matchings_in_induced_subgraph = count_matchings_simple(induced_subgraph)
#         # print(f"Matchings in induced subgraph at current node {current} is = {matchings_in_induced_subgraph}")

#         # count matchings in subgraph induced by nodes in vertices_encountered including current

#         # new_result = count_matchings_forcibly_saturate_vertex(induced_subgraph,introduced_element)
#         # print(f"count matchings included vertex: {current} result: {new_result} ")
#         # combined_result = new_result+sum_matchings_child_subgraph
#         new_states = {}
#         for state_string in child_states:
#           # print(f"STATE STRING IS: {state_string}")
#           # print("index of introduced element: ",index_of_introduced_element)
#           # Case where new node is NOT matched (append to string that new node is not matched and carry through the old value)
#           new_string_new_node_unsaturated = state_string[0:index_of_introduced_element] + "0" + state_string[index_of_introduced_element:]
#           # print(f"new node unsaturated. new string: {new_string_new_node_unsaturated} state_string: {state_string} child_states[state_string]: {child_states[state_string]}")
#           new_states[new_string_new_node_unsaturated] = child_states[state_string]



#           # Case where new node IS matched (append to string that new node is matched  and calculate in original graph)
#           new_string_new_node_saturated = state_string[0:index_of_introduced_element] + "1" + state_string[index_of_introduced_element:]
#           # print(f"new node sat. new string: {new_string_new_node_saturated} state_string: {state_string}")
#           # forcibly_saturate_result = count_matchings_forcibly_saturate_vertex(induced_subgraph,introduced_element)

#           child_elements_to_unsaturate = []
#           elements_to_saturate = [introduced_element]
#           print(f"Gonna construct child elements to unsaturate. statestring: {state_string}, child bag: {child_bag} els to sat: {elements_to_saturate} ")
#           for i in range(len(state_string)):
#             if state_string[i] == "0":
#               child_elements_to_unsaturate.append(child_bag[i])
#             else:
#               elements_to_saturate.append(child_bag[i])

#           forcibly_saturate_unsaturate_result = count_matchings_forcibly_saturate_and_unsaturate(induced_subgraph,elements_to_saturate,child_elements_to_unsaturate)
#           print(f"sat/unsat result for the subgraph: {induced_subgraph.nodes} w edges: {induced_subgraph.edges} where els to sat are: {elements_to_saturate} and to unsat are: {child_elements_to_unsaturate}, result: {forcibly_saturate_unsaturate_result}")
#           # print(f"Calling forcibly saturated on this subgraph w nodes: {induced_subgraph.nodes} and forcibly matching element {introduced_element}. Result is: {forcibly_saturate_result}") 
#           # print(f"SETTING NEW STRING NODE SATURATED {new_string_new_node_saturated} TO VALUE {forcibly_saturate_unsaturate_result}")
#           new_states[new_string_new_node_saturated] = forcibly_saturate_unsaturate_result
#         # print("COMBINED RESULT: ",combined_result)
#         print("Returning from introduce node. New states: ",new_states)
#         return new_states, vertices_encountered

#     else:
#       raise Exception("Ey, too many children: ", len(children))
#   # After while loop, we are now at a leaf
#   print("THink we are at leaf, current: ",current)
#   leaf_value = decomposition.nodes[current]["values"]
#   # print(f"Leaf value: {leaf_value}")
#   # vertices_encountered.update(set(leaf_value))
#   # print("after adding initial to v encountered: ", vertices_encountered)
#   # for state in possible_states:
#   #   computation = 1+1+1
#   #   state_results[state] = computation
#   # return vertices_encountered
#   leaf_state = {"0":1, "1":0}
#   print(f"Returning leaf state: {leaf_state} and eaf value: {leaf_value}")
#   return leaf_state, set(leaf_value)
    

# def entry_count(graph, decomposition, decomposition_root):
#   state_results, vertices_encountered = count_matchings_in_nice_tree_decomp_REMEMBER_STATE(graph,decomposition,decomposition_root,None)
#   number_of_matchings = 0
#   for key in state_results:
#     number_of_matchings += state_results[key]
#   return number_of_matchings

# Bit cleaner version

In [90]:
state_results = {}

def count_matchings_in_nice_tree_decomp_REMEMBER_STATE(graph, decomposition, current, parent):
  children = list(decomposition.neighbors(current))
  one_child_time_total = 0
  state_time_total = 0
  try:
    children.remove(parent)
  except ValueError:
    pass
    # print("Current element has no parent, must be root")
#   print(f"Current node: {current} has these children after removal of parent: {children}")
  while len(children) > 0:
    if len(children) == 2:
      assert len(children) == 2, f"Expected 2 children, got {len(children)}"
      assert type(children) == list, f"TYpe of children should be list, it is {children}"
      two_children_time = time.time_ns()
      child_states_left, vertices_encountered_left_set = count_matchings_in_nice_tree_decomp_REMEMBER_STATE(graph, decomposition,children[0],current)
      child_states_right, vertices_encountered_right_set = count_matchings_in_nice_tree_decomp_REMEMBER_STATE(graph, decomposition,children[1],current)
      if vertices_encountered_left_set.issubset(vertices_encountered_right_set) or vertices_encountered_right_set.issubset(vertices_encountered_left_set):
        # print(f"Ey whats going on, one subgraph is a susbet of the other? {vertices_encountered_left_set} and {vertices_encountered_right_set}")
        if len(vertices_encountered_left_set) > len(vertices_encountered_right_set):
          return child_states_left, vertices_encountered_left_set
        else:
          return child_states_right, vertices_encountered_right_set
    #   print("At join node. Current: ",current)
      vertices_encountered_left = list(vertices_encountered_left_set)
      vertices_encountered_right = list(vertices_encountered_right_set)
      vertices_encountered_left.sort()
      vertices_encountered_right.sort()
      current_bag = decomposition.nodes[current]["values"]
      left_child_bag = decomposition.nodes[children[0]]["values"]
      right_child_bag = decomposition.nodes[children[1]]["values"]
      current_bag.sort()
      left_child_bag.sort()
      right_child_bag.sort()


      new_states = {}
      # Iterates over all possible states at the join node, e.g. "000", "001", "010", "011", etc
      time_state_strings_start = time.time_ns()
      for state_string_tuple in list(itertools.product("01",repeat=len(current_bag))):
        state_string = "".join(state_string_tuple)
        # print("STATE STRING: ",state_string)
        # Find the indices in the string which contain a 1
        # e.g. "010" -> [1],          "011" -> [1,2]
        indices_of_ones = []
        for i in range(len(state_string)):
            if state_string[i] == "1":
                indices_of_ones.append(i)
        # print("INDIECS OF ONS: ",indices_of_ones)
        # Compute ways to distribute these ones to the two subgraphs,
        # e.g. [1] -> (([1],),(,[1])     [1,2] -> (([1,2],),([1],[2]),([2],[1]),(,[1,2]))
        state_inner_accumulator = 0
        for indices_of_ones_assigned_to_left in list(powerset(indices_of_ones,0,len(state_string)+1)):
            indices_of_ones_assigned_to_right = list(set(indices_of_ones)-set(indices_of_ones_assigned_to_left))
            # Now, for e.g. the state string "011" and the index example [1,2] -> ([1],[2]) 
            # this can be interpreted as "match index 1 of the state string in the left subgraph, match index 2 of the state string in the right subgraph"
            # I must now fill the "rest" of the state string indices with 0's in order to forcibly unsaturate them
            # then, I will obtain these signatures: subgraph1: "010" subgraph2: "001"
            # print(f"indiced of ones assigned to left: {indices_of_ones_assigned_to_left} and right: {indices_of_ones_assigned_to_right}")
            state_string_left = ["0" for x in range(len(state_string))]
            for index_of_one_to_left in indices_of_ones_assigned_to_left:
                state_string_left[int(index_of_one_to_left)] = "1"
                
            state_string_right = ["0" for x in range(len(state_string))]
            for index_of_one_to_right in indices_of_ones_assigned_to_right:
                state_string_right[int(index_of_one_to_right)] = "1"
            # print(f"statestringleft: {state_string_left} and statesright: {state_string_right}")

            # print(f"Now ready to go, I think. join bag: {current_bag} bags left, right: {vertices_encountered_left} and {vertices_encountered_right}")
            # # print(f"And graph string left right: {graph_string_left} and {graph_string_right}")
            # print()
            # print(f"state string left graph: {state_string_left}")
            # print(f"state string right graph: {state_string_right}")
            # print()


            left_value = child_states_left["".join(state_string_left)]
            right_value = child_states_right["".join(state_string_right)]
            combined_value = left_value*right_value
            state_inner_accumulator += combined_value
        #     print("left value: ",left_value)
        #     print("right value: ",right_value)


        # print(f"")
        # print(f"For the state {state_string}, state inner accumulator= {state_inner_accumulator}")
        new_states[state_string] = state_inner_accumulator
        time_state_strings_stop = time.time_ns()
        # print()
        # print()
        # print(f"Time spent handling iterating over all state strings at join node: {(time_state_strings_start-time_state_strings_stop)/1000}")
        state_time_total += (time_state_strings_start-time_state_strings_stop)
      vertices_encountered = vertices_encountered_left_set.union(vertices_encountered_right_set)
    #   print(f"Returning from introduce node {current}. New states: {new_states}")
      two_children_time_stop = time.time_ns()
      print(f"two children time: {(two_children_time_stop-two_children_time)}")
      return new_states, vertices_encountered
    elif len(children) == 1:
      one_child_time_start = time.time_ns()
      assert len(children) == 1, f"Expected 1 child, got {len(children)}"
      child_states, vertices_encountered = count_matchings_in_nice_tree_decomp_REMEMBER_STATE(graph, decomposition, children[0],current)
      # print(f"One child case. Current: {current} and child_states: {child_states}")

      child_bag = decomposition.nodes[children[0]]["values"]
      current_bag = decomposition.nodes[current]["values"]
      child_bag.sort()
      current_bag.sort()
    #   print(f"Child values: {child_bag}")
    #   print(f"Current values: {current_bag}")
      if len(child_bag) > len(current_bag):
        # print(f"We must be at a forget node. aggregating. Current node: {current} and child: {children[0]} ")
        forgotten_element = list(set(child_bag) - set(current_bag))
        assert len(forgotten_element) == 1, f"Forgotten element set should only contain one value, but it is: {forgotten_element}"
        forgotten_element = forgotten_element[0]
        index_of_forget_element = child_bag.index(forgotten_element)
        new_states = {}
        for state_string in child_states:
        #   print("In forget node looping over child states. Current child state string: ",state_string)
          new_string = state_string[:index_of_forget_element] + state_string[index_of_forget_element+1:]
          try:
            new_states[new_string] = new_states[new_string] + child_states[state_string]
          except KeyError:
             new_states[new_string] = child_states[state_string]


        # print("Returning these new states from forget node: ", new_states)
        return new_states, vertices_encountered
      else:
        # print("we must be at introduce node")
        introduced_element = list(set(current_bag) - set(child_bag))
        assert len(introduced_element) == 1, f"introduced element should be 1 element but is: {introduced_element}"
        introduced_element = introduced_element[0]
        index_of_introduced_element = current_bag.index(introduced_element)
        # print(f"Index of the element {introduced_element} within current values {current_bag} is {index_of_introduced_element}")
        vertices_encountered.update(set(decomposition.nodes[current]["values"]))
        induced_subgraph = graph.subgraph(vertices_encountered)
        # print(f"Induced subgraph nodes: {induced_subgraph.nodes}")
        # print(f"Induced subgraph edges: {induced_subgraph.edges}")
        new_states = {}
        for state_string in child_states:
          # Case where new node is NOT matched (append to string that new node is not matched and carry through the old value)
          new_string_new_node_unsaturated = state_string[0:index_of_introduced_element] + "0" + state_string[index_of_introduced_element:]
          new_states[new_string_new_node_unsaturated] = child_states[state_string]



          # Case where new node IS matched (append to string that new node is matched  and calculate in original graph)
          new_string_new_node_saturated = state_string[0:index_of_introduced_element] + "1" + state_string[index_of_introduced_element:]

          child_elements_to_unsaturate = []
          elements_to_saturate = [introduced_element]
        #   print(f"Gonna construct child elements to unsaturate. statestring: {state_string}, child bag: {child_bag} els to sat: {elements_to_saturate} ")
          for i in range(len(state_string)):
            if state_string[i] == "0":
              child_elements_to_unsaturate.append(child_bag[i])
            else:
              elements_to_saturate.append(child_bag[i])
          time_count = time.time()
          forcibly_saturate_unsaturate_result = count_matchings_forcibly_saturate_and_unsaturate(induced_subgraph,elements_to_saturate,child_elements_to_unsaturate)
          time_count_stop = time.time()
          time_total = time_count_stop-time_count
          if time_total > 0.5:
            print(f"Time spent forcibly saturating counting: {time_total} seconds for node {current}")
        #   print(f"sat/unsat result for the subgraph: {induced_subgraph.nodes} w edges: {induced_subgraph.edges} where els to sat are: {elements_to_saturate} and to unsat are: {child_elements_to_unsaturate}, result: {forcibly_saturate_unsaturate_result}")
          new_states[new_string_new_node_saturated] = forcibly_saturate_unsaturate_result
        # print("Returning from introduce node. New states: ",new_states)
        one_child_time_stop = time.time_ns()
        one_child_time_total += one_child_time_stop-one_child_time_start
        # print(f"One child time spent: {round((one_child_time_stop-one_child_time_start)/1000)}")
        return new_states, vertices_encountered
    else:
      raise Exception("Ey, too many children: ", len(children))
  # After while loop, we are now at a leaf
#   print("THink we are at leaf, current: ",current)
  leaf_value = decomposition.nodes[current]["values"]
  leaf_state = {"0":1, "1":0}
#   print(f"Returning leaf state: {leaf_state} and eaf value: {leaf_value}")
  print(f"one child divide by 1000: {one_child_time_total}")
  print(f"state total div by 1000: {state_time_total}")
  return leaf_state, set(leaf_value)
    

def entry_count(graph, decomposition, decomposition_root):
  state_results, vertices_encountered = count_matchings_in_nice_tree_decomp_REMEMBER_STATE(graph,decomposition,decomposition_root,None)
  number_of_matchings = 0
  for key in state_results:
    number_of_matchings += state_results[key]
  return number_of_matchings

In [91]:
forget_graph = nx.Graph()
forget_graph.add_nodes_from(["a","b","c"])
forget_graph.add_edge("a","b")
forget_graph.add_edge("b","c")

In [92]:
forget_decomp = nx.Graph()
forget_decomp.add_node(0,values=["a"])
forget_decomp.add_node(1,values=["a","b"])
forget_decomp.add_node(2,values=["b"])
forget_decomp.add_node(3,values=["b","c"])
forget_decomp.add_node(4,values=["c"])
forget_decomp.add_edge(0,1)
forget_decomp.add_edge(1,2)
forget_decomp.add_edge(2,3)
forget_decomp.add_edge(3,4)

In [93]:
join_graph = nx.Graph()
join_graph.add_nodes_from(["a","b","c","d"])
join_graph.add_edge("a","b")
join_graph.add_edge("b","c")
join_graph.add_edge("c","d")

join_decomp = nx.Graph()
join_decomp.add_node(0,values=["a"])
join_decomp.add_node(1,values=["a","b"])
join_decomp.add_node(2,values=["b"])
join_decomp.add_node(3,values=["b","c"])
join_decomp.add_node(4,values=["b","c"])
join_decomp.add_node(5,values=["b","c"])
join_decomp.add_node(6,values=["c"])
join_decomp.add_node(7,values=["c","d"])
join_decomp.add_node(8,values=["d"])
join_decomp.add_edge(0,1)
join_decomp.add_edge(1,2)
join_decomp.add_edge(2,3)
join_decomp.add_edge(3,4)
join_decomp.add_edge(4,5)
join_decomp.add_edge(5,6)
join_decomp.add_edge(6,7)
join_decomp.add_edge(7,8)

In [94]:
count_matchings_in_nice_tree_decomp_REMEMBER_STATE(join_graph,join_decomp,4,None)

one child divide by 1000: 0
state total div by 1000: 0
powerset length is: 0

FOR THIS SATURATION DIRECTION INSTANCE ('a', 'b')
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 1

FOR THIS SATURATION DIRECTION INSTANCE ('b', 'c')
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
one child divide by 1000: 0
state total div by 1000: 0
powerset length is: 0

FOR THIS SATURATION DIRECTION INSTANCE ('d', 'c')
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 1

FOR THIS SATURATION DIRECTION INSTANCE ('c', 'b')
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Tim

({'00': 1, '01': 1, '10': 1, '11': 3}, {'a', 'b', 'c', 'd'})

In [95]:
# count_matchings_in_nice_tree_decomp_REMEMBER_STATE(forget_graph,forget_decomp2,4,None)

In [96]:
pcm

In [97]:
count_matchings_in_nice_tree_decomp_REMEMBER_STATE(pcm,pcm_decomp,pcm_decomp_root,None)

one child divide by 1000: 0
state total div by 1000: 0
powerset length is: 0

FOR THIS SATURATION DIRECTION INSTANCE (0, 1)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 0

FOR THIS SATURATION DIRECTION INSTANCE (1, 3)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
one child divide by 1000: 0
state total div by 1000: 0
powerset length is: 0

FOR THIS SATURATION DIRECTION INSTANCE (7, 8)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 1

FOR THIS SATURATION DIRECTION INSTANCE (7, 6)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is

({'': 17}, {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10})

In [98]:
forget_decomp.nodes(data=True)

NodeDataView({0: {'values': ['a']}, 1: {'values': ['a', 'b']}, 2: {'values': ['b']}, 3: {'values': ['b', 'c']}, 4: {'values': ['c']}})

In [99]:
pcm_decomp.nodes(data=True)

NodeDataView({0: {'values': [3, 4]}, 1: {'values': [6, 7, 9]}, 2: {'values': [1, 2]}, 3: {'values': [0, 1]}, 4: {'values': [5, 6, 10]}, 5: {'values': [7, 8]}, 6: {'values': [4, 5, 10]}, 7: {'values': [1, 3]}, 8: {'values': [6, 9, 10]}, 9: {'values': [4]}, 10: {'values': [3]}, 11: {'values': [6, 9]}, 12: {'values': [7]}, 13: {'values': [0, 1, 3]}, 14: {'values': [5, 10]}, 15: {'values': [6, 10]}, 16: {'values': [1]}, 17: {'values': [0]}, 18: {'values': [7]}, -1: {'values': []}, 19: {'values': [0, 1, 3]}, 20: {'values': [0, 1, 3]}, 21: {'values': [1]}, 22: {'values': [0, 1]}, 23: {'values': [4, 10]}, 24: {'values': [6, 7]}})

In [100]:
for e in pcm_decomp.nodes(data=True):
    print(e)

(0, {'values': [3, 4]})
(1, {'values': [6, 7, 9]})
(2, {'values': [1, 2]})
(3, {'values': [0, 1]})
(4, {'values': [5, 6, 10]})
(5, {'values': [7, 8]})
(6, {'values': [4, 5, 10]})
(7, {'values': [1, 3]})
(8, {'values': [6, 9, 10]})
(9, {'values': [4]})
(10, {'values': [3]})
(11, {'values': [6, 9]})
(12, {'values': [7]})
(13, {'values': [0, 1, 3]})
(14, {'values': [5, 10]})
(15, {'values': [6, 10]})
(16, {'values': [1]})
(17, {'values': [0]})
(18, {'values': [7]})
(-1, {'values': []})
(19, {'values': [0, 1, 3]})
(20, {'values': [0, 1, 3]})
(21, {'values': [1]})
(22, {'values': [0, 1]})
(23, {'values': [4, 10]})
(24, {'values': [6, 7]})


In [101]:
for e in pcm_graph.nodes(data=True):
    print(e)

(0, {'values': [3, 4]})
(1, {'values': [6, 7, 9]})
(2, {'values': [1, 2]})
(3, {'values': [0, 1]})
(4, {'values': [5, 6, 10]})
(5, {'values': [7, 8]})
(6, {'values': [4, 5, 10]})
(7, {'values': [1, 3]})
(8, {'values': [6, 9, 10]})
(9, {'values': [4]})
(10, {'values': [3]})
(11, {'values': [6, 9]})
(12, {'values': [7]})
(13, {'values': [0, 1, 3]})
(14, {'values': [5, 10]})
(15, {'values': [6, 10]})
(16, {'values': [1]})
(17, {'values': [0]})
(18, {'values': [7]})
(-1, {'values': []})
(19, {'values': [0, 1, 3]})
(20, {'values': [0, 1, 3]})


# Okay... 19 apr kl 19:15: Ser ud til at jeg nogenlunde har løst det med den der simple abcd path graf som eksempel (men den tæller 1 matching for meget konsekvent.. det må jeg lige kigge på).
# det virker ikke med PCM, der sniger sig tuples ind i states fra child. check det ud.

In [102]:
p5 = nx.path_graph(5)
p5_decomp, p5_root = construct_nice_decomposition(p5)
p6 = nx.path_graph(6)
p6_decomp, p6_root = construct_nice_decomposition(p6)
p7 = nx.path_graph(7)
p7_decomp, p7_root = construct_nice_decomposition(p7)

Current root: -1 has no parent, is probably root of entire tree
Current root: -1 has no parent, is probably root of entire tree
Current root: -1 has no parent, is probably root of entire tree


In [103]:
p5_decomp.nodes(data=True)

NodeDataView({0: {'values': [3, 4]}, 1: {'values': [0, 1]}, 2: {'values': [2, 3]}, 3: {'values': [1, 2]}, 4: {'values': [3]}, 5: {'values': [1]}, 6: {'values': [2]}, 7: {'values': [3]}, 8: {'values': [0]}, -1: {'values': []}})

In [104]:
count_matchings_in_nice_tree_decomp_REMEMBER_STATE(p5,p5_decomp,p5_root,None)

one child divide by 1000: 0
state total div by 1000: 0
powerset length is: 0

FOR THIS SATURATION DIRECTION INSTANCE (0, 1)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 1

FOR THIS SATURATION DIRECTION INSTANCE (1, 2)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 2

FOR THIS SATURATION DIRECTION INSTANCE (2, 3)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 3

FOR THIS SATURATION DIRECTION INSTANCE (3, 4)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0


({'': 5}, {0, 1, 2, 3, 4})

In [105]:
count_matchings_in_nice_tree_decomp_REMEMBER_STATE(p6,p6_decomp,p6_root,None)

one child divide by 1000: 0
state total div by 1000: 0
powerset length is: 0

FOR THIS SATURATION DIRECTION INSTANCE (0, 1)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 1

FOR THIS SATURATION DIRECTION INSTANCE (1, 2)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 2

FOR THIS SATURATION DIRECTION INSTANCE (2, 3)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 3

FOR THIS SATURATION DIRECTION INSTANCE (3, 4)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
p

({'': 6}, {0, 1, 2, 3, 4, 5})

In [106]:
count_matchings_in_nice_tree_decomp_REMEMBER_STATE(p7,p7_decomp,p7_root,None)

one child divide by 1000: 0
state total div by 1000: 0
powerset length is: 0

FOR THIS SATURATION DIRECTION INSTANCE (5, 6)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 1

FOR THIS SATURATION DIRECTION INSTANCE (5, 4)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 2

FOR THIS SATURATION DIRECTION INSTANCE (4, 3)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 3

FOR THIS SATURATION DIRECTION INSTANCE (3, 2)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
p

({'': 7}, {0, 1, 2, 3, 4, 5, 6})

In [107]:
halicin

In [108]:
halicin_decomp, halicin_root = construct_nice_decomposition(halicin)

Current root: -1 has no parent, is probably root of entire tree


In [109]:
count_matchings_in_nice_tree_decomp_REMEMBER_STATE(halicin,halicin_decomp,halicin_root,None)

one child divide by 1000: 0
state total div by 1000: 0
powerset length is: 0

FOR THIS SATURATION DIRECTION INSTANCE (9, 11)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 1

FOR THIS SATURATION DIRECTION INSTANCE (9, 10)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 1

FOR THIS SATURATION DIRECTION INSTANCE (9, 8)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 2

FOR THIS SATURATION DIRECTION INSTANCE (8, 7)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0

({'': 25}, {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14})

In [110]:
count_matchings_simple(halicin)

1146

In [111]:
268+627

895

In [112]:
aspirin_graph = read_smiles("O=C(C)Oc1ccccc1C(=O)O")
aspirin_decomp, aspirin_root = construct_nice_decomposition(aspirin_graph)

Current root: -1 has no parent, is probably root of entire tree


In [113]:
count_matchings_simple(aspirin_graph)

335

In [114]:
count_matchings_in_nice_tree_decomp_REMEMBER_STATE(aspirin_graph,aspirin_decomp,aspirin_root,None)

one child divide by 1000: 0
state total div by 1000: 0
powerset length is: 0

FOR THIS SATURATION DIRECTION INSTANCE (0, 1)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0013284683227539062
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 0

FOR THIS SATURATION DIRECTION INSTANCE (1, 3)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
one child divide by 1000: 0
state total div by 1000: 0
powerset length is: 0

FOR THIS SATURATION DIRECTION INSTANCE (10, 12)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 0

FOR THIS SATURATION DIRECTION INSTANCE (10, 11)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0

({'': 40}, {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12})

In [115]:
87+215

302

In [116]:
benzene_graph = read_smiles("c1ccccc1")
benzene_decomp, benzene_root = construct_nice_decomposition(benzene_graph)

Current root: -1 has no parent, is probably root of entire tree


In [117]:
count_matchings_in_nice_tree_decomp_REMEMBER_STATE(benzene_graph,benzene_decomp,benzene_root,None)

one child divide by 1000: 0
state total div by 1000: 0
powerset length is: 0

FOR THIS SATURATION DIRECTION INSTANCE (4, 3)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 0

FOR THIS SATURATION DIRECTION INSTANCE (2, 3)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 1

FOR THIS SATURATION DIRECTION INSTANCE (4, 5)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 2

FOR THIS SATURATION DIRECTION INSTANCE (4, 3, 5)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 

({'': 11}, {0, 1, 2, 3, 4, 5})

In [118]:
count_matchings_simple(benzene_graph)

18

In [119]:
cyclopropane_graph = read_smiles("c1cc1")
cyclopropane_decomp, cyclopropane_root = construct_nice_decomposition(cyclopropane_graph)

Current root: -1 has no parent, is probably root of entire tree


In [120]:
g, r = construct_junction_and_add_leaves(cyclopropane_graph)
# handle_promiscuous_nodes2(g,r,None)
# handle_join_nodes(g,r,None)
# print(f"In entry point. Root: {root}")
# make_decomposition_nice(g,r, None)

In [121]:
g,r = construct_nice_decomposition(cyclopropane_graph)

Current root: -1 has no parent, is probably root of entire tree


In [122]:
print(g.nodes(data=True))
print(g.edges)
print(r)

[(0, {'values': [0, 1, 2]}), (1, {'values': [0]}), (2, {'values': [1]}), (-1, {'values': []}), (3, {'values': [0, 1]}), (4, {'values': [0, 2]})]
[(0, 3), (0, 4), (1, 4), (2, -1), (2, 3)]
-1


In [123]:
remapped = remap_graph(junction_tree(cyclopropane_graph))

In [124]:
g.nodes(data=True)

NodeDataView({0: {'values': [0, 1, 2]}, 1: {'values': [0]}, 2: {'values': [1]}, -1: {'values': []}, 3: {'values': [0, 1]}, 4: {'values': [0, 2]}})

In [125]:
g.edges

EdgeView([(0, 3), (0, 4), (1, 4), (2, -1), (2, 3)])

In [126]:
count_matchings_simple(cyclopropane_graph)

4

In [127]:
count_matchings_in_nice_tree_decomp_REMEMBER_STATE(benzene_graph,benzene_decomp,benzene_root,None)

one child divide by 1000: 0
state total div by 1000: 0
powerset length is: 0

FOR THIS SATURATION DIRECTION INSTANCE (4, 3)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 0

FOR THIS SATURATION DIRECTION INSTANCE (2, 3)
make power set, add stuff to saturation edges in instance etc: 0.0012063980102539062
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 1

FOR THIS SATURATION DIRECTION INSTANCE (4, 5)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 2

FOR THIS SATURATION DIRECTION INSTANCE (4, 3, 5)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating e

({'': 11}, {0, 1, 2, 3, 4, 5})

In [128]:
count_matchings_simple(benzene_graph)

18

In [129]:
cyclopentane_graph = read_smiles("c1cccc1")
cyclopentane_decomp, cyclopentane_root = construct_nice_decomposition(cyclopentane_graph)

Current root: -1 has no parent, is probably root of entire tree


In [130]:
count_matchings_simple(cyclopentane_graph)

11

In [131]:
count_matchings_in_nice_tree_decomp_REMEMBER_STATE(cyclopentane_graph,cyclopentane_decomp,cyclopentane_root,None)

one child divide by 1000: 0
state total div by 1000: 0
powerset length is: 0

FOR THIS SATURATION DIRECTION INSTANCE (4, 3)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 0

FOR THIS SATURATION DIRECTION INSTANCE (2, 3)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 1

FOR THIS SATURATION DIRECTION INSTANCE (2, 1)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 2

FOR THIS SATURATION DIRECTION INSTANCE (2, 1, 3)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 

({'': 9}, {0, 1, 2, 3, 4})

In [132]:
print(cyclopentane_graph.nodes)
print(cyclopentane_graph.edges)

[0, 1, 2, 3, 4]
[(0, 1), (0, 4), (1, 2), (2, 3), (3, 4)]


In [133]:
print(cyclopentane_decomp.nodes(data=True))
print(cyclopentane_decomp.edges)
print(cyclopentane_root)

[(0, {'values': [0, 1, 4]}), (1, {'values': [2, 3, 4]}), (2, {'values': [1, 2, 4]}), (3, {'values': [1, 4]}), (4, {'values': [2, 4]}), (5, {'values': [0]}), (6, {'values': [2]}), (-1, {'values': []}), (7, {'values': [0, 1]}), (8, {'values': [2, 4]})]
[(0, 3), (0, 7), (1, 4), (1, 8), (2, 3), (2, 4), (5, -1), (5, 7), (6, 8)]
-1


In [134]:
cyclobutane_graph = read_smiles("c1ccc1")
ccb_d, ccb_r = construct_nice_decomposition(cyclobutane_graph)

Current root: -1 has no parent, is probably root of entire tree


In [135]:
count_matchings_simple(cyclobutane_graph)

7

In [136]:
count_matchings_in_nice_tree_decomp_REMEMBER_STATE(cyclobutane_graph,ccb_d,ccb_r,None)

one child divide by 1000: 0
state total div by 1000: 0
powerset length is: 0

FOR THIS SATURATION DIRECTION INSTANCE (0, 3)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 0

FOR THIS SATURATION DIRECTION INSTANCE (0, 1)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 1

FOR THIS SATURATION DIRECTION INSTANCE (1, 2)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 1

FOR THIS SATURATION DIRECTION INSTANCE (3, 2)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
p

({'': 7}, {0, 1, 2, 3})

In [137]:
ccb_d.nodes(data=True)

NodeDataView({0: {'values': [1, 2, 3]}, 1: {'values': [0, 1, 3]}, 2: {'values': [1, 3]}, 3: {'values': [1]}, 4: {'values': [0]}, -1: {'values': []}, 5: {'values': [1, 2]}, 6: {'values': [0, 3]}})

In [138]:
ccb_d.edges

EdgeView([(0, 2), (0, 5), (1, 2), (1, 6), (3, -1), (3, 5), (4, 6)])

In [139]:
ccb_r

-1

In [140]:
cyclobutane_graph.nodes

NodeView((0, 1, 2, 3))

In [141]:
cyclobutane_graph.edges

EdgeView([(0, 1), (0, 3), (1, 2), (2, 3)])

In [142]:
count_matchings_simple(cyclobutane_graph)

7

In [143]:
count_matchings_forcibly_saturate_and_unsaturate(cyclobutane_graph,[3,2,1],[])

powerset length is: 2

FOR THIS SATURATION DIRECTION INSTANCE (0, 1, 2)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 2

FOR THIS SATURATION DIRECTION INSTANCE (2, 3, 0)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0


2

In [144]:
count_matchings_vertex_included_multiple(cyclobutane_graph,[1,2,3])

powerset length is: 2

FOR THIS SATURATION DIRECTION INSTANCE (0, 3, 2)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 2

FOR THIS SATURATION DIRECTION INSTANCE (2, 1, 0)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0


2

In [145]:
count_matchings_forcibly_saturate_and_unsaturate(cyclopentane_graph,[0,4],[1])

powerset length is: 2

FOR THIS SATURATION DIRECTION INSTANCE (4, 0)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0


1

In [146]:
lst2 = (2,3,1)
lst = [1,2,3]

In [147]:
all(lst)

True

In [148]:
all(x in lst2 for x in lst)

True

In [149]:
sorted((1,2))

[1, 2]

In [150]:
nx.is_matching(cyclopentane_graph,[[0,4],[1,2]])

True

In [151]:
set({(0,4),(4,0)})

{(0, 4), (4, 0)}

In [152]:
count_matchings_in_nice_tree_decomp_REMEMBER_STATE(halicin,halicin_decomp,halicin_root,None)

one child divide by 1000: 0
state total div by 1000: 0
powerset length is: 0

FOR THIS SATURATION DIRECTION INSTANCE (9, 11)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 1

FOR THIS SATURATION DIRECTION INSTANCE (9, 10)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 1

FOR THIS SATURATION DIRECTION INSTANCE (9, 8)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 2

FOR THIS SATURATION DIRECTION INSTANCE (8, 7)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0

({'': 25}, {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14})

In [153]:
count_matchings_simple(halicin)

1146

In [154]:
count_matchings_in_nice_tree_decomp_REMEMBER_STATE(pcm,pcm_decomp,pcm_root,None)

one child divide by 1000: 0
state total div by 1000: 0
powerset length is: 0

FOR THIS SATURATION DIRECTION INSTANCE (0, 1)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 0

FOR THIS SATURATION DIRECTION INSTANCE (1, 3)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
one child divide by 1000: 0
state total div by 1000: 0
powerset length is: 0

FOR THIS SATURATION DIRECTION INSTANCE (7, 8)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 1

FOR THIS SATURATION DIRECTION INSTANCE (7, 6)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is

({'': 17}, {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10})

In [ ]:
entry_count(halicin,halicin_decomp,halicin_root)

1146

In [ ]:
entry_count(pcm,pcm_decomp,pcm_root)

140

In [ ]:
dct = {"a":1,"b":3}
for x in dct:
    print(x)
    print(dct[x])

a
1
b
3


In [150]:
lsd = read_smiles("CCN(CC)C(=O)[C@H]1CN([C@@H]2Cc3c[nH]c4c3c(ccc4)C2=C1)C")
lsd_decomp, lsd_root = construct_nice_decomposition(lsd)

Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@@H]" contains stereochemical information that will be discarded.


Current root: -1 has no parent, is probably root of entire tree


In [151]:
lsd_root

-1

In [152]:
start = time.time()
result = entry_count(lsd,lsd_decomp,lsd_root)
stop = time.time()
print(f"Time spent: {stop-start}")
print(result)

one child divide by 1000: 0
state total div by 1000: 0
powerset length is: 0

FOR THIS SATURATION DIRECTION INSTANCE (15, 20)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 0

FOR THIS SATURATION DIRECTION INSTANCE (20, 19)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 1

FOR THIS SATURATION DIRECTION INSTANCE (19, 18)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 2

FOR THIS SATURATION DIRECTION INSTANCE (19, 20, 18)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_e

In [431]:
t1 = time.time_ns()
t2 = time.time_ns()
t2-t1

0

In [153]:
entry_count(pcm,pcm_decomp,pcm_root)

one child divide by 1000: 0
state total div by 1000: 0
powerset length is: 0

FOR THIS SATURATION DIRECTION INSTANCE (0, 1)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 0

FOR THIS SATURATION DIRECTION INSTANCE (1, 3)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
one child divide by 1000: 0
state total div by 1000: 0
powerset length is: 0

FOR THIS SATURATION DIRECTION INSTANCE (7, 8)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is_matching: 0
Time spent creating edges_to_evaluate: 0
powerset length is: 1

FOR THIS SATURATION DIRECTION INSTANCE (7, 6)
make power set, add stuff to saturation edges in instance etc: 0.0
stuff after power set: 0.0
Time spent using is

17

# 20 apr 17:55: Jeg er kommet til at ødelægge den "clean"  `count_matchings_vertex_included_multiple`, den som indeholder timing. den giver forkert resultat. fiks det lige.
# Resultat fra timingeksperimenter: over 90% af den lange tid bruges i de mange, mange kald til `is_matching`. Jeg tror ikke, at `nx.is_matching` er langsom i sig selv, men den lange tid skyldes, at `powerset` er *ENORMT*! Jeg skaber alle combinations, også selvom subsets af dem er invalid. F.eks., hvis edgesettet {(1,2),(3,4)} ikke er valid, så behøver jeg ikke at undersøge {(1,2),(3,4),(5,6)} osv... fiks dette !